In [ ]:
import pandas as pd
import sys

#csv에서 news content를 txt로 변환

csv=pd.read_csv('../Data/2019/2019.csv',encoding='utf-8-sig')

text=csv['text']
print(text)

with open('News_Contents.txt','w',encoding='utf-8-sig') as file:
    for item in text:
        file.write("%s\n" % item)


In [ ]:
import pandas as pd

df = pd.read_csv('../Data/2019/2019.csv', encoding='utf-8-sig')
df.head()

In [ ]:
from kiwipiepy import Kiwi
import pandas as pd
import math

kiwi = Kiwi()
kiwi.load_user_dictionary('sorted_wordlist_end.txt')
kiwi.prepare()

stopword = []
def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if wordlist[1].startswith('N'):
            stopword.append(wordlist[0])
    stopwords = set(stopword)
    files.close()
    return stopword

def npmi(a, b, n, z):
    return (math.log((n*z)/(a*b))) / (math.log(n/z))

# def npmi3(a, b, c, n, z):
#     a = math.log((n*n*z)/(a*b*c))
#     b = 2*math.log(n/z)
#     return a / b

In [28]:
from kss import split_sentences
from collections import Counter
import pprint as pp
import re

rm_stopword('한국어불용어100.txt')
docs = []
merge_token = []
bi_gram = {}
i = 0
for news in open('News_Contents.txt', encoding='utf-8'):
    doc = []
    context = []
    com = []
    lines = split_sentences(news)

    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word.strip() for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
        context += content
        doc.append(content)

    #2 token npmi
    for line in doc:
        com += list(zip(line, line[1:]))
    nmi = {}
    n = len(context)
    cnt = Counter(context)
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    tmp = [key[0]+" "+key[1] for key, item in result if item > 0.89]

    #2 token N_GRAM 연결
    # tmp = []
    # for line in doc:
    #     tmp += list(zip(line, line[1:]))
    for word in tmp:
        if word in bi_gram:
            bi_gram[word] += 1
        else:
            bi_gram[word] = 1
    
    # bi_gram = sorted(bi_gram.items(), key = lambda item : item[1], reverse=True)
    # merge_token = {key for key, item in bi_gram if item > 2}
    # print(merge_token)

    
    # 토큰을 합치는 내용
    # for token in merge_token:
    #     for content in doc:
    #         for i in range(len(content)-1):
    #             cmp_str = '_'.join(content[i:i+2])
    #             if cmp_str == token:
    #                 content.pop(i)
    #                 content.pop(i)
    #                 content.insert(i, token)
    # docs.append(doc)
    i +=1 
    if i % 10000 == 0:
        print(i)
    # print(len(docs),"===================================")
    # if len(docs) % 10000 == 0:
    #     print(len(docs))

bi_gram = sorted(bi_gram.items(), key = lambda item : item[1], reverse=True)

with open('pmi_token.txt','w',encoding='utf-8-sig') as file:
    for key, value in bi_gram:
        file.write("%s" % (key+' , '+str(value)+"\n"))
        if value < 20:
            break

# df['N_list'] = docs
# df.to_csv('2019_pre.csv', encoding='utf-8-sig')

In [ ]:
import tomotopy as tp

def extract_topic(doc):
    model = tp.LDAModel(k=5, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    for line in doc:
        model.add_doc(line) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = []
    for j in range(0,100):
        model.train(10) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        print(k,res)
        for w, p in res:
            if w not in s.keys():
                s.append(w)

        # s = ', '.join(w for w, p in res)
    # print(s)
    return ', '.join(s)

In [ ]:
for doc in docs:
    print(extract_topic(doc))